# resize
This script is resize images to 440*440.

Change root

In [ ]:
import cv2
import numpy as np

def resize_with_padding_256(img, keypoints=None, target_size=440, pad_value=0):
    """
    將圖片等比例縮放，讓長邊變成 target_size，短邊再置中 padding 到 target_size。
    若有 keypoints，會同步做相同的縮放 + padding。

    參數：
        img: 輸入圖片 (H, W, C)
        keypoints: (可選) numpy array，shape = (N, 2)，每一列 [x, y]
        target_size: 最終輸出邊長（預設 256）
        pad_value: padding 顏色 (0=黑)

    回傳：
        若 keypoints 為 None: img_out
        否則: (img_out, kps_out)
    """
    h, w = img.shape[:2]
    long_side = max(h, w)
    if long_side == 0:
        raise ValueError("圖片尺寸有問題，高或寬為 0")

    # 等比例縮放比例
    scale = float(target_size) / float(long_side)
    new_w = max(int(round(w * scale)), 1)
    new_h = max(int(round(h * scale)), 1)

    # 先 resize
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # 計算 padding（置中）
    pad_left = pad_right = pad_top = pad_bottom = 0
    if new_w < target_size:
        pad_total = target_size - new_w
        pad_left = pad_total // 2
        pad_right = pad_total - pad_left
    if new_h < target_size:
        pad_total = target_size - new_h
        pad_top = pad_total // 2
        pad_bottom = pad_total - pad_top

    img_out = cv2.copyMakeBorder(
        resized,
        pad_top, pad_bottom, pad_left, pad_right,
        borderType=cv2.BORDER_CONSTANT,
        value=pad_value
    )

    # 處理 keypoints
    if keypoints is None:
        return img_out

    kps = keypoints.astype(np.float32).copy()
    x = kps[:, 0]
    y = kps[:, 1]

    # 先等比例縮放
    x_scaled = x * scale
    y_scaled = y * scale

    # 再加上 padding offset
    x_new = x_scaled + pad_left
    y_new = y_scaled + pad_top

    kps_out = np.stack([x_new, y_new], axis=1)
    return img_out, kps_out


In [ ]:
import os
import csv
import cv2
import numpy as np
from tqdm import tqdm

def process_dataset_to_256(
    img_dir,
    csv_in,
    img_out_dir,
    csv_out,
    target_size=440,
    pad_value=0,
):
    """
    將某一個資料夾內的所有骨架圖片 + 對應 keypoints
    統一轉成 target_size x target_size（等比例縮放 + 置中 padding），
    並輸出新的圖片 + CSV。

    假設：
        - img_dir 裡是骨架圖片（檔名像 000123.png）
        - csv_in 裡的 filename 欄位是「原始彩圖檔名」，例如 000123.jpg 或 000123.png
        - 兩者用「不含副檔名的 stem」對應
    """
    if not os.path.exists(img_dir):
        print(f"錯誤：找不到圖片目錄 {img_dir}")
        return

    os.makedirs(img_out_dir, exist_ok=True)

    # 1) 讀 CSV，依 stem 建索引
    if not os.path.exists(csv_in):
        print(f"錯誤：找不到 CSV {csv_in}")
        return

    with open(csv_in, "r", newline="") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        if header is None or "filename" not in header:
            raise ValueError("CSV 必須包含 'filename' 欄位")

        cols = header[1:]  # 除去 filename，其餘應為 x,y 成對
        if len(cols) % 2 != 0:
            raise ValueError("除了 filename 之外，CSV 欄位數應該是偶數 (x, y 成對)")

        # 建立 x,y 欄位對
        kp_pairs = []
        for i in range(0, len(cols), 2):
            x_col = cols[i]
            y_col = cols[i + 1]
            kp_pairs.append((x_col, y_col))

        rows_by_stem = {}
        for row in reader:
            fname = row["filename"]              # e.g. "000123.jpg" or "000123.png"
            stem = os.path.splitext(fname)[0]    # "000123"
            rows_by_stem[stem] = row

    # 2) 準備輸出 CSV
    with open(csv_out, "w", newline="") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=header)
        writer.writeheader()

        # 3) 找出所有圖片檔
        exts = [".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".tif"]
        img_files = [
            name for name in os.listdir(img_dir)
            if any(name.lower().endswith(ext) for ext in exts)
        ]

        print(f"處理資料夾 {img_dir}，共 {len(img_files)} 張圖片")

        for img_name in tqdm(img_files, desc="resize to 256"):
            img_path = os.path.join(img_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"警告：無法讀取圖片 {img_name}")
                continue

            stem, ext = os.path.splitext(img_name)   # 骨架圖片的 stem

            row = rows_by_stem.get(stem)
            if row is None:
                print(f"警告：找不到 stem={stem} 對應的 keypoints，略過 keypoints")
                # 只處理圖片，不寫 CSV 行
                img_256 = resize_with_padding_256(img, keypoints=None,
                                                 target_size=target_size,
                                                 pad_value=pad_value)
                out_path = os.path.join(img_out_dir, img_name)
                cv2.imwrite(out_path, img_256)
                continue

            # 把 row 的 keypoints 讀成 numpy array
            kp_list = []
            for x_col, y_col in kp_pairs:
                x_val = float(row[x_col])
                y_val = float(row[y_col])
                kp_list.append([x_val, y_val])
            kps = np.array(kp_list, dtype=np.float32)

            # 做 resize + padding
            img_256, kps_256 = resize_with_padding_256(
                img, keypoints=kps,
                target_size=target_size,
                pad_value=pad_value
            )

            # 存圖片
            out_path = os.path.join(img_out_dir, img_name)
            cv2.imwrite(out_path, img_256)

            # 存新的 keypoints 一行到 CSV
            new_row = dict(row)  # 保留原本的 filename，不動也可以
            # 如果你想把 filename 換成骨架圖檔名，可以改：
            # new_row["filename"] = img_name

            for idx, (x_col, y_col) in enumerate(kp_pairs):
                x_new, y_new = kps_256[idx]
                new_row[x_col] = float(x_new)
                new_row[y_col] = float(y_new)

            writer.writerow(new_row)

    print(f"完成：圖片輸出到 {img_out_dir}，CSV 輸出到 {csv_out}")


In [ ]:
root = ""

process_dataset_to_256(
    img_dir=root+"/sk/original_70",
    csv_in=root+"/sk/original_70.csv",
    img_out_dir=root+"/resized/original_70",
    csv_out=root+"/resized/resized_original_70.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/original",
    csv_in=root+"/sk/original.csv",
    img_out_dir=root+"/resized/original",
    csv_out=root+"/resized/resized_original.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/swirl_180",
    csv_in=root+"/sk/swirl_180.csv",
    img_out_dir=root+"/resized/swirl_180",
    csv_out=root+"/resized/resized_swirl_180.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/resize_left_0.7",
    csv_in=root+"/sk/resize_left_0.7.csv",
    img_out_dir=root+"/resized/resize_left_0.7",
    csv_out=root+"/resized/resized_resize_left_0.7.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/resize_left_1.5",
    csv_in=root+"/sk/resize_left_1.5.csv",
    img_out_dir=root+"/resized/resize_left_1.5",
    csv_out=root+"/resized/resized_resize_left_1.5.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/resize_right_0.5",
    csv_in=root+"/sk/resize_right_0.5.csv",
    img_out_dir=root+"/resized/resize_right_0.5",
    csv_out=root+"/resized/resized_resize_right_0.5.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/resize_right_1.3",
    csv_in=root+"/sk/resize_right_1.3.csv",
    img_out_dir=root+"/resized/resize_right_1.3",
    csv_out=root+"/resized/resized_resize_right_1.3.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/resize_upper_0.7",
    csv_in=root+"/sk/resize_upper_0.7.csv",
    img_out_dir=root+"/resized/resize_upper_0.7",
    csv_out=root+"/resized/resized_resize_upper_0.7.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/resize_upper_1.5",
    csv_in=root+"/sk/resize_upper_1.5.csv",
    img_out_dir=root+"/resized/resize_upper_1.5",
    csv_out=root+"/resized/resized_resize_upper_1.5.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/resize_lower_0.5",
    csv_in=root+"/sk/resize_lower_0.5.csv",
    img_out_dir=root+"/resized/resize_lower_0.5",
    csv_out=root+"/resized/resized_resize_lower_0.5.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/resize_lower_1.3",
    csv_in=root+"/sk/resize_lower_1.3.csv",
    img_out_dir=root+"/resized/resize_lower_1.3",
    csv_out=root+"/resized/resized_resize_lower_1.3.csv",
    target_size=256,
    pad_value=0,
)



In [ ]:
root = ""

process_dataset_to_256(
    img_dir=root+"/sk/swirl_15",
    csv_in=root+"/sk/swirl_15.csv",
    img_out_dir=root+"/resized/swirl_15",
    csv_out=root+"/resized/resized_swirl_15.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/swirl_30",
    csv_in=root+"/sk/swirl_30.csv",
    img_out_dir=root+"/resized/swirl_30",
    csv_out=root+"/resized/resized_swirl_30.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/swirl_45",
    csv_in=root+"/sk/swirl_45.csv",
    img_out_dir=root+"/resized/swirl_45",
    csv_out=root+"/resized/resized_swirl_45.csv",
    target_size=256,
    pad_value=0,
)

process_dataset_to_256(
    img_dir=root+"/sk/swirl_60",
    csv_in=root+"/sk/swirl_60.csv",
    img_out_dir=root+"/resized/swirl_60",
    csv_out=root+"/resized/resized_swirl_60.csv",
    target_size=256,
    pad_value=0,
)